In [2]:
!pip3 install streamlit

In [1]:
import pandas as pd
import re
import pymorphy2

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [3]:
import os
 
# выводим пути к папкам (dirpath) и наименования файлов (filenames) и после этого
for dirpath, _, filenames in os.walk('.'):
  
  # во вложенном цикле проходимся по названиям файлов
  for filename in filenames:
 
    # и соединяем путь до папок и входящие в эти папки файлы
    # с помощью метода path.join()
    print(os.path.join(dirpath, filename))

.\l2_text_tfidf.ipynb
.\test.csv
.\train.csv
.\Задание2.ipynb
.\.ipynb_checkpoints\l2_text_tfidf-checkpoint.ipynb
.\.ipynb_checkpoints\Задание2-checkpoint.ipynb


In [5]:
df = pd.read_csv('train.csv')

In [6]:
df.head()

,text,label
0,На нас обиделась Оля....Жизнь потеряла смысл. ...,0
1,RT @PolinaZuzy: http://t.co/ZYwbUowIti единств...,1
2,"@AHamueva Машину назвать малышкой,мать ты ебу ...",1
3,RT @malina23_: отличный день в школе!:*теперь ...,1
4,@Serebryakovaaa не то слово))) у него что ни м...,1


In [7]:
df['text'] = df['text'].str.replace('/','')

In [8]:
# функция удаления пунктуации и цифр
def remove_trash(list): 
    pattern = '[^А-Яа-я0-9]+'
    try:
      list = [re.sub(pattern, ' ', i) for i in list] 
      print (list)
    except Exception as e:
      print(e)
    return list

In [9]:
df['text_clean'] = remove_trash(df['text'])

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [10]:
df = df[~df.text_clean.isna()].copy()

In [11]:
df.shape

(136100, 3)

In [12]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk

In [13]:
stop_russian = stopwords.words('russian')

In [14]:
len(stop_russian)

151

In [15]:
text_transformer = TfidfVectorizer(stop_words=stop_russian, ngram_range=(1,1), lowercase=True, max_features=15000 )

In [16]:
df.tail()

,text,label,text_clean
136095,А Зоя и Рита Шабановы все-таки сучки. Р.S. М...,1,А Зоя и Рита Шабановы все таки сучки Р Маринка...
136096,@nvarlygina я все пропустила!:( ну хоть ты заг...,0,я все пропустила ну хоть ты загадала и надеюс...
136097,"RT @milkiwey_: выходные были хорошими, даже яр...",1,выходные были хорошими даже яркими очень
136098,"@leshqa_meow ахахах, ну ты же стааарше меня, м...",1,ахахах ну ты же стааарше меня мне нужна подде...
136099,"Пришла к прабабушке, она мне говорит с кем ты ...",0,Пришла к прабабушке она мне говорит с кем ты т...


In [17]:
text = text_transformer.fit_transform(df['text_clean'])

In [19]:
df.text.value_counts()

Офигенный день!\nдень позитива)\nбегал как идиот целый день!\nтанцы офигенны, хоть я и ракал но мне очень понравилось!                                                  96
RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретвит((((( RT                                                                                                                 62
sm&amp;gt; напомни, как называется: фотка, вокруг черная рамка, а внизу - надпись?\nau&amp;gt; некролог?\nsm&amp;gt; не.. (\nau&amp;gt; демотиватор?\nsm&amp;gt; ДА!    43
RT @Creative_Link: Дождались! Официальный трейлер "Иван Царевич и Серый Волк - 2". Смотреть всем: http:t.cogmelSGMIuW Толи еще будет)                                   42
RT @Podslyshano: люблю зиму за то, что люди носом разблокируют телефоны, и такие оглядываются типа без палева)) а я то все вижу. да да.                                 35
                                                                                                                                                 

In [21]:
X_train, X_test, y_train, y_test = train_test_split(text, df['text'], test_size=0.3, random_state=1912)

In [22]:
print(X_train.shape,X_test.shape)

(95270, 15000) (40830, 15000)


In [23]:
from sklearn import tree

In [24]:
clf = tree.DecisionTreeClassifier()

In [ ]:
clf.fit(X_train,y_train)

In [ ]:
pred = clf.predict(X_test)

In [ ]:
import sklearn

In [ ]:
print('Качество модели по метрике F1', sklearn.metrics.f1_score(y_test,pred,average='macro'))

In [ ]:
morph = pymorphy2.MorphAnalyzer()

In [ ]:
%%time #првоерить время выполнения функции
# функция лемматизации - приводит в 1 форму
def lemmatize(row):
    t = []
    text = row['text_clean']
    for word in text.split():
        if len(word)<=3:   #удаляет слова, которые меньше == 3 символа
            continue
        p = morph.parse(word)[0]
        t.append(p.normal_form)
    return " ".join(t)

In [ ]:
%%time
df['text_clean_normal'] = df.apply(lemmatize,axis=1)

In [ ]:
df[['text_clean','text_clean_normal']]

### качество модели после лемматизации

In [ ]:
text_norm = text_transformer.fit_transform(df['text_clean_normal'])
X_train, X_test, y_train, y_test = train_test_split(text_norm, df['topic_label'], test_size=0.3, random_state=1912)

In [ ]:
clf_norm = tree.DecisionTreeClassifier()
clf_norm.fit(X_train,y_train)
pred_norm = clf_norm.predict(X_test)
print('Качество модели по метрике F1 после лемматизации', sklearn.metrics.f1_score(y_test,pred_norm,average='macro'))